In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import string
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split

In [108]:
#lines = []
#with open('Video_Games_5.json') as f:
#     for line in f:
#            lines.append(line.strip())

In [109]:
#write_lines = '[' + ', '.join(lines) + ']'

In [110]:
#with open('Video_Games_5_vec.json', 'w') as f:
#    f.write(write_lines)

In [111]:
amazon = pd.read_json("./Video_Games_5_vec.json")

In [112]:
amazon.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0700099867,"[8, 12]",1,Installing the game was a struggle (because of...,"07 9, 2012",A2HD75EMZR8QLN,123,Pay to unlock content? I don't think so.,1341792000
1,0700099867,"[0, 0]",4,If you like rally cars get this game you will ...,"06 30, 2013",A3UR8NLLY1ZHCX,"Alejandro Henao ""Electronic Junky""",Good rally game,1372550400
2,0700099867,"[0, 0]",1,1st shipment received a book instead of the ga...,"06 28, 2014",A1INA0F5CWW3J4,"Amazon Shopper ""Mr.Repsol""",Wrong key,1403913600
3,0700099867,"[7, 10]",3,"I got this version instead of the PS3 version,...","09 14, 2011",A1DLMTOTHQ4AST,ampgreen,"awesome game, if it did not crash frequently !!",1315958400
4,0700099867,"[2, 2]",4,I had Dirt 2 on Xbox 360 and it was an okay ga...,"06 14, 2011",A361M14PU2GUEG,"Angry Ryan ""Ryan A. Forrest""",DIRT 3,1308009600


In [113]:
amazon_reviews = amazon[['reviewText', 'overall']]

In [114]:
punc_map = str.maketrans(dict.fromkeys(string.punctuation))
num_map = str.maketrans(dict.fromkeys(string.digits))
word_counts = {}
for review in amazon_reviews['reviewText']:
    prep_review = review.strip().lower().translate(punc_map).translate(num_map).split()
    for word in prep_review:
        word_counts[word] = word_counts.get(word, 0) + 1 

In [115]:
word_count_series = pd.Series(word_counts)

In [116]:
features = list(word_count_series.sort_values(ascending=False)[50:1050].index)

In [117]:
amazon_reviews_s = amazon_reviews.sample(frac=.025)

In [118]:
len(amazon_reviews_s)

5794

In [119]:
review_text = amazon_reviews_s['reviewText'].copy()
for key in features[:500]:
    amazon_reviews_s['word_' + str(key)] = review_text.str.contains(' ' + str(key) + ' ', case=False)

In [120]:
amazon_reviews_s.head()

,reviewText,overall,word_good,word_when,word_great,word_time,word_fun,word_me,word_which,word_about,...,word_titles,word_longer,word_drive,word_absolutely,word_stop,word_gamer,word_truly,word_learn,word_maps,word_stick
142928,this controller is great and works just fine.....,5,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
31514,As a fan of NASCAR if you say you only enjoy t...,5,False,True,False,False,True,False,True,False,...,False,False,True,False,False,False,False,False,False,False
19869,This game is a great game for my whole family!...,5,False,False,True,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
45960,Does anybody know anything about this game.Ple...,5,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
185403,It looks like rockstar has done it again with ...,5,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [121]:
amazon_reviews_s['overall'] = amazon_reviews_s['overall'].astype('uint8')
amazon_reviews_s['good?'] = amazon_reviews_s['overall'] >= 3

In [123]:
amazon_reviews_model = amazon_reviews_s.drop(['overall', 'reviewText'], axis=1).copy()

In [124]:
X_train, X_test, Y_train, Y_test = train_test_split(amazon_reviews_model, amazon_reviews_model['good?'], test_size=.25, random_state=42)

In [125]:
bnb = BernoulliNB()
bnb.fit(X_train, Y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [130]:
bnb.score(X_test, Y_test)

y_pred = bnb.predict(X_test)
true_pos = ((Y_test == 1) & (y_pred == 1)).sum()
true_neg = ((Y_test == 0) & (y_pred == 0)).sum()
false_pos = ((Y_test == 0) & (y_pred == 1)).sum()
false_neg = ((Y_test == 1) & (y_pred == 0)).sum()

specificity = true_neg / (true_neg + false_pos)
sensitivity = true_pos / (true_pos + false_neg)

print("Confusion Matrix: ")
print(true_neg, false_neg)
print(false_pos, true_pos)

Confusion Matrix: 
175 1
0 1273
